In [1]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Activation, Embedding, Flatten, GlobalMaxPool1D, Dropout, Conv1D
from keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from keras.losses import binary_crossentropy
from keras.optimizers import Adam
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import keras
import json

import firebase_admin
from firebase_admin import credentials
from firebase_admin import db
from firebase_admin import firestore


Using TensorFlow backend.


In [2]:
model = keras.models.load_model('model-conv1d.h5')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


In [3]:
with open("categories.json","r") as fp:
    categories = json.load(fp)
encoder = LabelEncoder()
encoder.fit(categories)

LabelEncoder()

### Read unlabeled events from Firestore

In [4]:
cred = credentials.Certificate('evently-key.json')
firebase_admin.initialize_app(cred)
store = firestore.client()
collection_name = "testEvents"

In [5]:
doc_ref = store.collection(collection_name)
try:
    docs = doc_ref.get()
except google.cloud.exceptions.NotFound:
    print(u'No such collection!')
data = []
for doc in docs:
    # print(u'{} => {}'.format(doc.id, doc.to_dict()))
    data.append(doc)

/Users/pangbo/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:3: DeprecationWarning: 'Collection.get' is deprecated:  please use 'Collection.stream' instead.
  app.launch_new_instance()


In [6]:
def predict(X,model,encoder, top_classes=1):
    tokenizer = Tokenizer(num_words=5000, lower=True)
    tokenizer.fit_on_texts(X)
    sequences = tokenizer.texts_to_sequences(X)
    X = pad_sequences(sequences, maxlen=180, padding="post")
    y_pred = model.predict(X)
    top_y = (-y_pred).argsort(axis=-1)[:,:top_classes]
    return [encoder.inverse_transform(y) for y in top_y]

### Preprocessing text and utilize trained model to predict

In [9]:
X = []
docId = []
for doc in data:
    x = doc.to_dict()['eventName']
    if 'description' in doc.to_dict() and doc.to_dict()['description']:
        x+=' '+doc.to_dict()['description']
    X.append(x)
    docId.append(doc.id)
predictedLabel = predict(X,model,encoder,top_classes = 3)
pairs = zip(predictedLabel,docId)
print len(pairs)

738
738


### Write back to Firestore

In [17]:
for categories,docId in pairs:
    print docId
    event_ref = store.collection(collection_name).document(docId)
    event_ref.set({
        u'categories': categories.tolist()
    }, merge=True)
print "Done"

Eventbrite31806907316
Eventbrite42211737443
